In [4]:
import numpy as np
from tqdm.notebook import tqdm
import cvxpy as cp

# Linear Models with Logarithmic Loss

Consider the following protocol: for $t=1,2,...,T$, nature chooses $x_t \in \mathbb{R}^d$ in each round, and then reveals a noisy label $y_t \sim \text{Ber}(x_t^\top \theta^\star) - 1/2$, where $\theta^\star \in \mathbb{R}^d$ is the unknown parameter. Our goal is to learn to make accurate predictions given a linear model class and any $x \in \mathbb{R}^d$, i.e., to predict $y^\star(x) = x^\top \theta^\star$. The classical way of learning $\theta^\star$ given i.i.d samples of $\{x_t,y_t\}_{t=1}^T$ is to perform ridge regression.

Ridge regression considers the following estimator and prediction: given $\{x_t,y_t\}_{t=1}^T$ compute:

$$
\hat\theta_T := \arg\min_{\theta \in \mathbb{R}^d} \sum_{t=1}^T \left( x_t^\top \theta - y_i\right)^2 + 
\lVert \theta \rVert^2
$$

and set $\hat y_T(x) := x^\top \hat\theta_T$. This is simply least-sqaures regression with $\ell_2$ regularization. The estimator and the prediciton can also be solved in closed form. Define $V_t := I + \sum_{t=1}^T x_t x_t^\top$, then we have:

$$
\hat y_T(x) = x^\top V_t^{-1} \sum_{t=1}^T x_t y_t.
$$

Another way of learning $\theta^\star$  given i.i.d samples of $\{x_t,y_t\}_{t=1}^T$ is to perform linear regression with the negative logarithmic (cross-entropy) loss. 

Linear regression with negative logarithmic loss considers the following estimator and prediciton: given $\{x_t,y_t\}_{t=1}^T$ compute:

$$
\hat\theta_T := \arg\max_{\theta \in \mathbb{R}^d} \mathcal{L}(\{x_t,y_t\}_{t=1}^T;\theta) = \arg\max_{\theta \in \mathbb{R}^d}  \sum_{t=1}^T y_t \log(x_t^\top \theta) + (1-y_t)\log(1-x_t^\top \theta)
$$
and set $\hat y_T(x) := x^\top \hat\theta_T$. The estimaotr and the prediciton cannot be solved in closed form, however the loss function defined above is concave in its argument. Therefore approximate solvers like Newtown's method or Gradient Descent, should be sufficient for finding the $\theta$ that minimizes the negative logarithmic loss. Taking the first derivative of the loss $\mathcal{L}$ we get:2*

$$
\frac{\partial}{\partial \theta}  \mathcal{L}(\{x_t,y_t\}_{t=1}^T;\theta) = g_T(\theta) :=  \sum_{t=1}^T \left(\frac{y_t}{x_t^\top \theta} - \frac{1-y_t}{1-x_t^\top \theta}\right) x_t - 2\theta.
$$

Taking the second derivative of $\mathcal{L}$ gives:

$$
\frac{\partial^2}{\partial \theta^2}  \mathcal{L}(\{x_t,y_t\}_{t=1}^T;\theta)= H_T(\theta) :=  \sum_{t=1}^T -\left(\frac{y_t}{(x_t^\top\theta)^2} + \frac{1-y_t}{(1-x_t^\top\theta)^2}\right) x_t x_t^\top - I.
$$

Notice that we are trying to solve for the zeros of $g_T(\theta)$. This function has three zeros at $\theta = \{-\infty, \arg\max_{\theta \in \mathbb{R}^d} \mathcal{L}(\{x_t,y_t\}_{t=1}^T;\theta), \infty\}$. Therefore, we need to constraint our optimization routine otherwise our estimate of $\theta$ may diverge.

One idea is to use Frank-Wolfe to ensure $\theta$ does not diverge off to infinity. 





Thus we can solve:

$$
\hat\theta_T := \arg\min_{\theta \in \mathbb{R}^d} - \mathcal{L}(\{x_t,y_t\}_{t=1}^T;\theta)
$$

by applying Newton's method for $k=1,2,...,n$, i.e.

$$
\theta_{k+1} = \theta_k - \left(H_T^{-1}(\theta_k)\right)^\top g_t(\theta_k).
$$

where $\theta_0$ is initialized apprioately, i.e. $\theta_0 \sim \mathcal{N}(0,I)$. Setting $\hat\theta_T = \theta_n$ gives us our estimator and letting $\hat y_T(x) = x^\top \hat\theta_T$ gives us our predictor. 

In [43]:
d = 2
num_features = 4
theta_star = np.random.uniform(size=d)
theta_star = theta_star / (10*np.linalg.norm(theta_star))
X = np.random.uniform(size=(num_features,d))
for i in range(num_features):
    X[i] = X[i] / np.linalg.norm(X[i])
X[0] = X[0] / 1000
X[1] = X[1] / 900
X[2] = X[2] / 800
X[3] = X[3] / 700


In [44]:
num_samples = 1000**2
features = np.zeros((num_samples,d))
tar = np.zeros(num_samples)
for i in tqdm(range(num_samples)):
    j = i % num_features
    feature = X[j]
    mean = np.inner(X[j],theta_star)
    obs = np.random.binomial(1,p=mean) - 1/2 
    features[i] = feature
    tar[i] = obs

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [57]:
class LinearLogRegression(object):
    def __init__(self, features, obs, d, fw_iters,theta_star,lam_max):
        self.features = features
        self.obs = obs
        self.n = len(obs)
        self.fw_iters = fw_iters
        self.d = d
        self.theta_star = theta_star
        self.t = 1 / (4 * lam_max)
    
    
    def get_gradient(self):
        grad = np.zeros(self.d)
        inner = np.dot(self.features,self.theta)
        mu = np.zeros((self.n,self.d))
        
        mu[:,0] = -4 * (inner - self.obs) / (4 * inner ** 2 - 1)
        for i in range(1,self.d):
            mu[:,i] = mu[:,0]
        mult = np.multiply(mu,self.features)
        grad = np.sum(mult,axis=0)
        
        return grad
    
    def frank_wolfe(self):
        self.theta = np.zeros(self.d) + 1/self.d
        for k in tqdm(range(self.fw_iters)):
            grad = self.get_gradient()
            norm = np.linalg.norm(grad)
            if norm != 0:
                s = - grad / (np.linalg.norm(grad)) * self.t 
            else:
                s = np.zeros(self.d)
            dt = s - self.theta
            gamma = np.inner(dt,grad) / (400 * np.linalg.norm(dt)**2)
            alpha = min(gamma,1)
            self.theta = self.theta + alpha * (s + self.theta)
            if k % 1000 == 999:
                print('Log Loss:' , np.linalg.norm(self.theta - self.theta_star))
        
            
    
    
    def Least_Squares(self):
        A = np.zeros((self.d,self.d))
        b = np.zeros(self.d)
        for i in range(self.n):
            x = self.features[i]
            y = self.obs[i] + 1 / 2
            A = A + np.outer(x,x)
            b = b + y * x
        self.theta_ls = np.linalg.solve(A,b)
        print('Least Squares:' , np.linalg.norm(self.theta_ls - self.theta_star))
    
            
        
        

In [58]:
num_data = 100000
eigs = np.linalg.eigvals(np.dot(X.T,X))
reg = LinearLogRegression(features[:num_data],tar[:num_data],d,50000,theta_star,1)
reg.Least_Squares()
print(np.dot(X,reg.theta_ls))
reg.frank_wolfe()

Least Squares: 0.051004004779794984
[1.14201934e-04 1.31492634e-04 7.24820315e-05 3.19138331e-05]


  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [47]:
reg.theta

array([0.14600185, 0.20293708])

In [48]:
reg.theta_ls

array([0.11812182, 0.01871451])

In [49]:
np.dot(X,theta_star)

array([9.91230135e-05, 1.06497475e-04, 1.00958322e-04, 8.32247336e-05])

In [50]:
np.dot(X,reg.theta)

array([0.00022075, 0.00022194, 0.00030079, 0.00029619])

In [30]:
np.dot(X.T,X)

array([[1.96463584e-06, 2.51572850e-06],
       [2.51572850e-06, 3.87324839e-06]])

In [7]:
num_data = 50000
x,y = features[:num_data],tar[:num_data]

In [21]:
inner = np.dot(x,theta_star)
mu = np.zeros((num_data,2))
mu[:,0] = -4 * (inner - y) / (4 * inner ** 2 - 1)
mu[:,1] = -4 * (inner - y) / (4 * inner ** 2 - 1)

In [24]:
v = np.multiply(mu,x)

In [25]:
np.shape(v)

(50000, 2)

In [28]:
np.sum(v,axis=1)

array([0.00225153, 0.00314318, 0.00297006, ..., 0.00314318, 0.00297006,
       0.00403398])